In [35]:
import pandas as pd
import numpy as np
import mathis as mt
from sklearn.decomposition import PCA

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline

from skopt.space import Integer, Real, Categorical
from skopt import BayesSearchCV
from scipy.stats import uniform, loguniform

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

from sklearn.metrics import roc_auc_score
import pickle

In [2]:
bio = pd.read_csv('data/train.csv')

- Train/Test split

In [3]:
x_list = bio.columns.tolist()
x_list.remove('Activity')
X = bio[x_list]
y = bio['Activity']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42, stratify = y, test_size = .35)

- Logistic Regression

In [4]:
logreg_pipe = Pipeline([
    ('pca', PCA(random_state = 42)),
    ('logreg', LogisticRegression(random_state = 42, solver = 'liblinear', penalty = 'l1'))
])

logreg_params = {
    'pca__n_components': Integer(300, 900),
    'logreg__tol': Real(0, .1, prior='uniform'),
    'logreg__C': Real(0.0001, 100, prior='log-uniform'),
    'logreg__class_weight': Categorical(['balanced', None]),
    'logreg__max_iter': Integer(0, 1000),
    'logreg__l1_ratio': Real(0, 1, prior='uniform')
}

logreg_bs = BayesSearchCV(estimator = logreg_pipe,
                     search_spaces = logreg_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     n_jobs = 8,
                     cv = 5,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [5]:
logreg_bs.fit(X_train, y_train)

C:\Users\Avanyali\AppData\Roaming\Python\Python37\site-packages\sklearn\linear_model\_logistic.py:1317: UserWarning: l1_ratio parameter is only used when penalty is 'elasticnet'. Got (penalty=l1)
  "(penalty={})".format(self.penalty))


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                        ('logreg',
                                         LogisticRegression(penalty='l1',
                                                            random_state=42,
                                                            solver='liblinear'))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='roc_auc',
              search_spaces={'logreg__C': Real(low=0.0001, high=100, prior='log-uniform', transform='identity'),
                             'logreg__class_we...egories=('balanced', None), prior=None),
                             'logreg__l1_ratio': Real(low=0, high=1, prior='uniform', transform='identity'),
                             'logreg__max_iter': Integer(low=0, high=1000, prior='uniform', transform='identity'),
                             'logreg__tol': Real(low=0, high=0.1, prior='unifor

In [6]:
logreg = logreg_bs.best_estimator_['logreg']

In [9]:
log_pca = logreg_bs.best_estimator_['pca']

In [11]:
pickle.dump(logreg, open('models/LogisticRegression.sav', 'wb'))

In [12]:
pickle.dump(log_pca, open('models/PCA_LogReg.sav', 'wb'))

- XGBoost

In [10]:
bio['Activity'].value_counts(normalize = True)

1    0.542255
0    0.457745
Name: Activity, dtype: float64

In [14]:
xgb_pipe = Pipeline([
    ('pca', PCA(random_state = 42)),
    ('xgb', XGBClassifier(base_score = .542255, random_state = 42, booster = 'gbtree', use_label_encoder = False, verbosity = 0))
])

xgb_params = {
    'pca__n_components' : Integer(300, 1000),
    'xgb__n_estimators': Integer(1, 100),
    'xgb__max_depth': Integer(1, 100),
    'xgb__max_delta_step': Real(0, .5),
    'xgb__min_child_weight': Real(0, 2, prior='uniform'),
    'xgb__learning_rate': Real(0, .2, prior='uniform')
}

xgb_bs = BayesSearchCV(estimator = xgb_pipe,
                     search_spaces = xgb_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     n_jobs = 8,
                     cv = 5,
                     verbose = 1,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [16]:
xgb_bs.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   12.0s remaining:   18.0s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   12.1s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   30.5s remaining:   45.8s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   30.9s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   14.0s remaining:   21.1s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   14.3s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   12.6s remaining:   18.9s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   12.8s finished


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                        ('xgb',
                                         XGBClassifier(base_score=0.542255,
                                                       booster='gbtree',
                                                       colsample_bylevel=None,
                                                       colsample_bynode=None,
                                                       colsample_bytree=None,
                                                       gamma=None, gpu_id=None,
                                                       importance_type='gain',
                                                       interaction_constraints=None,
                                                       learning_rate=None,
                                                       max_delta_step=None,
                                                       max_depth=None,
           

In [17]:
xgb = xgb_bs.best_estimator_['xgb']

In [18]:
xgb_pca = xgb_bs.best_estimator_['pca']

In [19]:
pickle.dump(xgb, open('models/XGBoost.sav', 'wb'))

In [20]:
pickle.dump(xgb_pca, open('models/PCA_XGB.sav', 'wb'))

- KNN

In [27]:
knn_pipe = Pipeline([
    ('pca', PCA(random_state = 42)),
    ('knn', KNeighborsRegressor(weights = 'distance'))
])

knn_params = {
    'pca__n_components' : Integer(300, 1000),
    'knn__n_neighbors': Integer(1, 13),
    'knn__p': Integer(1, 3)
}

knn_bs = BayesSearchCV(estimator = knn_pipe,
                     search_spaces = knn_params,
                     scoring = 'roc_auc',
                     n_iter = 50,
                     n_jobs = 8,
                     cv = 5,
                     verbose = 1,
                     refit = True,
                     optimizer_kwargs = {'base_estimator': 'RF'},
                     random_state=42)

In [28]:
knn_bs.fit(X_train, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:    5.3s remaining:    8.0s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:    5.4s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   12.0s remaining:   18.0s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   12.6s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


...

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   15.4s remaining:   23.2s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   16.0s finished


Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done   2 out of   5 | elapsed:   16.0s remaining:   24.1s
[Parallel(n_jobs=8)]: Done   5 out of   5 | elapsed:   16.4s finished


BayesSearchCV(cv=5,
              estimator=Pipeline(steps=[('pca', PCA(random_state=42)),
                                        ('knn',
                                         KNeighborsRegressor(weights='distance'))]),
              n_jobs=8, optimizer_kwargs={'base_estimator': 'RF'},
              random_state=42, scoring='roc_auc',
              search_spaces={'knn__n_neighbors': Integer(low=1, high=13, prior='uniform', transform='identity'),
                             'knn__p': Integer(low=1, high=3, prior='uniform', transform='identity'),
                             'pca__n_components': Integer(low=300, high=1000, prior='uniform', transform='identity')},
              verbose=1)

In [29]:
knn = knn_bs.best_estimator_['knn']

In [30]:
xgb_pca = xgb_bs.best_estimator_['pca']

In [31]:
pickle.dump(xgb, open('models/KNN.sav', 'wb'))

In [32]:
pickle.dump(xgb_pca, open('models/PCA_KNN.sav', 'wb'))

- Model scoring

In [37]:
xgb_bs.score(X_train, y_train), xgb_bs.score(X_test, y_test)

(0.9476196189112835, 0.7997099871001515)

In [38]:
logreg_bs.score(X_train, y_train), logreg_bs.score(X_test, y_test)

(0.8839504064114868, 0.8283548953990539)

In [39]:
knn_bs.score(X_train, y_train), knn_bs.score(X_test, y_test)

(1.0, 0.8013808913982314)